In [1]:
import os
import string
#import argparse

import numpy as np
import tensorflow as tf

/Users/florianmuellerklein/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/florianmuellerklein/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading saved models for inference

Loading a saved inference graph is very similar to loading up a saved session. Breaking down the procedure into steps we have:

1. Load the graph definition with tensorflow
2. Make the loaded graph the default graph for the session
3. Retrieve the input placeholders and output tensors from the graph
4. Push data through the graph using a `feed_dict` like normal

In this example we will build a pig latin translation program using the saved inference graph for the encoder-decoder network. 

### Load the inference graph

Tensorflow provides file I/O capabilities within its `gfile` module. We can use that or the standard `with open('filename.pb', 'rb') as f:` methodology. They should both work for our case. If you happen to be using 'Google Cloud Storage' then you will have to use `gfile` rather than the conventional python methods. 

After reading the graph definition file we will load it in a way that allows us to interact with it by using `tf.import_graph_def()`. Which takes the parsed graph definition as it's first argument and a name. The name will be a prefix that is added to each of the names in the graph. For example, if we had something called 'outputs' in our original graph it will now be called 'name/outputs'. 

In [2]:
with tf.gfile.GFile('./saved_translator/piglatin_enc-dec.pb', 'rb') as f:
    graph_definition = tf.GraphDef()
    graph_definition.ParseFromString(f.read())
    
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_definition, name='encoder-decoder')

NotFoundError: ./saved_translator/piglatin_enc-dec.pb; No such file or directory